<a href="https://colab.research.google.com/github/Zann181/Parcial-3_SyS/blob/main/Parcial_3_Dashboards_Streamlit_Ngrok_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía Streamlit en Colab y Ngrok - Dashboards

Elaborado por: Juan David Muñoz Buritica jmunozbu@unal.edu.co (Monitoria SyS 2024-1)
Revisado: Andrés Marino Álvarez Meza amalvarezme@unal.edu.co

- [Streamlit](https://streamlit.io/) es una librería destinada a crear y compartir aplicaciones o dashboards.
- Su objetivo es ser una herramienta fácil de usar que permita ejecutar scripts directamente y desplegarlos en un aplicativo web.
- Sin embargo, antes de ver su funcionamiento, debemos tener en cuenta que, si bien Streamlit permite correr el aplicativo, debemos alojarlo y "hacerlo visible" a la red.
- Existen diferentes métodos que cumplen dicho objetivo, también dependiendo del nivel de madurez que se requiera, bien sea simplemente testeo o producción. - Una alternativa eficaz, rápida y que puede ser fácilmente escalable es [Ngrok](https://ngrok.com/).

- Ngrok es un servicio y aplicativo globalmente distribuido que asegura, acelera y protege las aplicaciones y servicios de red.
- En este caso, nos servirá para crear el tunel http que nos deje visualizar el dashboard corriendo con Streamlit desde el entorno del cuadernillo o script de Python.
- El servicio gratuito de Ngrok, tras crear la cuenta tiene la posibilidad de mantener un agente activo simultáneo, cuya configuración está dada por el token de autenticación único por usuario y túnel.

**Nota: Cree una cuenta en Ngrok https://ngrok.com/ con su correo UNAL y guarde el token personal asignado**

In [3]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [1]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [4]:
# Instalar ffmpeg si es necesario en Colab
!apt-get install ffmpeg

import os
import streamlit as st
from yt_dlp import YoutubeDL

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


La estructura básica para correr un programa o script en streamlit es la siguiente.

- A continuación se muestra una app básica con algunas de las funcionalidades básicas para visualizar en dashboard.

- Debemos crear un archivo .py con los códigos del dashboard para su posterior visualización en Ngrok.

In [5]:
%%writefile parcial3_dashboard.py
import streamlit as st
import numpy as np
from numpy.fft import rfftfreq
import matplotlib.pyplot as plt
from scipy.signal import freqz, bilinear, lfilter, tf2zpk

import io
import soundfile as sf
from scipy.signal import resample, lfilter, bilinear
from scipy.fft import rfft
from scipy.special import softmax
import subprocess
import os
import joblib

import matplotlib.pyplot as plt
from scipy import signal

# Función para crear el sistema masa-resorte-amortiguador y obtener las respuestas
def simular_sistema_masa_resorte(m, c, k):
    # Crear la función de transferencia con scipy (numerador y denominador)
    numerator = [1]  # El numerador es 1 en la función de transferencia dada
    denominator = [m, c, k]  # Corresponde a ms^2 + cs + k

    # Crear el sistema de función de transferencia
    system = signal.TransferFunction(numerator, denominator)

    # Generar el diagrama de Bode
    w, mag, phase = signal.bode(system)

    # Generar la respuesta al escalón
    t_step, step_response = signal.step(system)

    # Generar la respuesta al impulso
    t_impulse, impulse_response = signal.impulse(system)

    return w, mag, phase, t_step, step_response, t_impulse, impulse_response

# Función para calcular los valores equivalentes del circuito RLC
def calcular_equivalente_RLC(m, c, k):
    L = m  # La inductancia es equivalente a la masa
    R = c  # La resistencia es equivalente al coeficiente de amortiguamiento
    C = 1 / k  # La capacitancia es la inversa de la constante del resorte
    return L, R, C

# Función para graficar los resultados
def graficar_resultados(w, mag, phase, t_step, step_response, t_impulse, impulse_response):
    # Crear las gráficas para Bode, escalón e impulso
    plt.figure(figsize=(14, 8))

    # Gráfico de magnitud Bode
    plt.subplot(2, 2, 1)
    plt.semilogx(w, mag)
    plt.title('Diagrama de Magnitud Bode')
    plt.xlabel('Frecuencia [rad/s]')
    plt.ylabel('Magnitud [dB]')
    plt.grid(True)

    # Gráfico de fase Bode
    plt.subplot(2, 2, 2)
    plt.semilogx(w, phase)
    plt.title('Diagrama de Fase Bode')
    plt.xlabel('Frecuencia [rad/s]')
    plt.ylabel('Fase [grados]')
    plt.grid(True)

    # Gráfico de respuesta al escalón
    plt.subplot(2, 2, 3)
    plt.plot(t_step, step_response)
    plt.title('Respuesta al Escalón')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.grid(True)

    # Gráfico de respuesta al impulso
    plt.subplot(2, 2, 4)
    plt.plot(t_impulse, impulse_response)
    plt.title('Respuesta al Impulso')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.grid(True)

    plt.tight_layout()
    st.pyplot(plt)





# Clase para manejar la descarga de audio
class AudioDownloader:
    def __init__(self, link="https://www.youtube.com/watch?v=51cZsDsVOqc"):
        self.link = link
        self.audio_mp3 = 'audio.mp3'
        self.audio_wav = 'output.wav'

    def download_audio(self):
        """Descarga y convierte el archivo de YouTube en formato wav"""
        # Eliminar archivos si ya existen
        if os.path.exists(self.audio_mp3):
            os.remove(self.audio_mp3)
        if os.path.exists(self.audio_wav):
            os.remove(self.audio_wav)

        st.write("Descargando audio desde YouTube...")
        subprocess.run(['yt-dlp', '--extract-audio', '-o', self.audio_mp3, '--audio-format', 'mp3', self.link], check=True)
        subprocess.run(['ffmpeg', '-i', self.audio_mp3, self.audio_wav], check=True)
        st.write("Audio descargado y convertido a formato wav.")

    def load_audio(self, target_fs=48000):
        """Carga el archivo de audio en formato wav y ajusta la frecuencia de muestreo si es necesario"""
        self.x, self.fs = sf.read(self.audio_wav)
        if len(self.x.shape) == 2:
            self.x = np.mean(self.x, axis=1)  # Convertir estéreo a mono
        st.write(f'Frecuencia de muestreo original: {self.fs} Hz')

        # Remuestreo si la frecuencia de muestreo no es la deseada
        if self.fs != target_fs:
            st.write(f"Remuestreando el audio a {target_fs} Hz...")
            num_samples = int(len(self.x) * target_fs / self.fs)
            self.x = resample(self.x, num_samples)
            self.fs = target_fs
            st.write(f"Remuestreo completo. Nueva frecuencia de muestreo: {self.fs} Hz")

        return self.x, self.fs

    def segment_audio(self, start_time, end_time):
        """Segmenta el audio entre los tiempos dados"""
        st.write(f"Segmentando el audio de {start_time}s a {end_time}s.")
        x_segment = self.x[int(start_time * self.fs):int(end_time * self.fs)]
        return x_segment, self.fs


# Clase para cargar y reconocer el género musical
class AudioRecognizer:
    def __init__(self, model):
        """Cargar el modelo previamente cargado"""
        self.model = model
        self.fmax = self.model['fmax']
        self.Xw_ = self.model['Xw_']
        self.vf = self.model['vf']
        self.label = self.model['label']
        self.name_c = self.model['name_c']
        self.fs = self.model['fs']
        st.write("Modelo cargado exitosamente.")

    def process_segment(self, audio_segment, fs):
        """Procesa el segmento de audio y calcula su espectro de frecuencias"""
        st.write("Calculando la transformada de Fourier del segmento de audio...")
        # Remuestrear si es necesario
        if fs != self.fs:
            st.write(f"Remuestreando el segmento a {self.fs} Hz...")
            num_samples = int(len(audio_segment) * self.fs / fs)
            audio_segment = resample(audio_segment, num_samples)
            fs = self.fs

        # Calcular la transformada rápida de Fourier (rfft) para señal real
        Xw_segment = np.abs(rfft(audio_segment)[:self.fmax])
        Xw_segment /= np.max(Xw_segment)  # Normalizar
        return Xw_segment

    def recognize_genre(self, processed_segment):
        """Reconoce si la canción es metal o reguetón basándose en el espectro de frecuencias"""
        st.write("Comparando el espectro del segmento con las canciones del modelo...")
        distances_metal = []
        distances_reggaeton = []

        for i, Xw_ref in enumerate(self.Xw_):
            dist = np.linalg.norm(processed_segment - Xw_ref[:self.fmax])  # Distancia euclidiana

            if self.label[i] == 1:  # Género 1: Metal
                distances_metal.append(dist)
            elif self.label[i] == 2:  # Género 2: Reguetón
                distances_reggaeton.append(dist)

        min_dist_metal = np.min(distances_metal)
        min_dist_reggaeton = np.min(distances_reggaeton)

        distances = np.array([min_dist_metal, min_dist_reggaeton])
        probabilities = softmax(-distances)  # Aplicamos softmax a las distancias negativas

        if probabilities[0] > probabilities[1]:
            genre = "Metal"
        else:
            genre = "Reguetón"

        return genre, probabilities

# Clase para realizar la modulación AM
class AMModulator:
    def __init__(self, audio, fs, carrier_freq=10000, mod_index=1.0):
        self.audio = audio
        self.fs = fs
        self.Ac = 1.0  # Amplitud de la portadora
        self.Fc = carrier_freq  # Frecuencia de la portadora
        self.mod_index = mod_index  # Índice de modulación ajustable
        self.t = np.linspace(0, len(self.audio) / self.fs, len(self.audio), endpoint=False)  # Vector de tiempo

    def generate_carrier(self, theta0=0):
        """Genera la señal portadora con fase θ0"""
        self.carrier = self.Ac * np.cos(2 * np.pi * self.Fc * self.t + theta0)
        return self.carrier

    def modulate_am(self):
        """Realiza la modulación AM"""
        self.generate_carrier()  # Generar la señal portadora
        message_norm = self.audio / np.max(np.abs(self.audio))  # Normalizar la señal de mensaje
        self.modulated_signal = (1 + self.mod_index * message_norm) * self.carrier
        return self.modulated_signal

    def demodulate_am(self):
        """Demodula la señal AM y aplica un filtro pasa bajas"""
        received_signal = self.modulated_signal * self.carrier  # Multiplicar por la portadora

        # Filtro pasa bajas (usando la Transformada Bilineal)
        wc = 2 * np.pi * (self.Fc / 2) / self.fs
        b_analog = [wc]
        a_analog = [1, wc]
        self.b_digital, self.a_digital = bilinear(b_analog, a_analog, self.fs)
        self.demodulated_signal = lfilter(self.b_digital, self.a_digital, received_signal)
        self.demodulated_signal *= (2 / self.Ac)  # Escalar la señal de demodulación
        return self.demodulated_signal

    def plot_signals(self):
        """Genera gráficos de las señales en el dominio del tiempo"""
        plt.figure(figsize=(14, 10))

        # Señal de Mensaje
        plt.subplot(4, 1, 1)
        plt.plot(self.t, self.audio)
        plt.title('Señal de Mensaje (m(t))')
        plt.xlabel('Tiempo [s]')
        plt.ylabel('Amplitud')

        # Señal Portadora
        plt.subplot(4, 1, 2)
        plt.plot(self.t, self.carrier)
        plt.title(f'Señal Portadora (c(t)) con frecuencia {self.Fc} Hz')
        plt.xlabel('Tiempo [s]')
        plt.ylabel('Amplitud')

        # Señal Modulada AM
        plt.subplot(4, 1, 3)
        plt.plot(self.t, self.modulated_signal)
        plt.title(f'Señal Modulada AM con índice {self.mod_index}')
        plt.xlabel('Tiempo [s]')
        plt.ylabel('Amplitud')

        # Señal Demodulada AM
        plt.subplot(4, 1, 4)
        plt.plot(self.t, self.demodulated_signal)
        plt.title('Señal Demodulada AM')
        plt.xlabel('Tiempo [s]')
        plt.ylabel('Amplitud')

        plt.tight_layout()
        st.pyplot(plt)

    def plot_frequency_domain(self, signal, title="Espectro de Frecuencia"):
        """Genera la gráfica del espectro de frecuencia"""
        fft_data = rfft(signal)
        freqs = rfftfreq(len(signal), 1/self.fs)

        plt.figure(figsize=(10, 4))
        plt.plot(freqs, np.abs(fft_data))
        plt.title(title)
        plt.xlabel('Frecuencia [Hz]')
        plt.ylabel('Magnitud')
        plt.grid()
        st.pyplot(plt)

    def save_audio_signals(self):
        """Guarda las señales de mensaje, modulada y demodulada en buffers para reproducir en Streamlit"""
        audio_buffer_message = io.BytesIO()
        audio_buffer_modulated = io.BytesIO()
        audio_buffer_demodulated = io.BytesIO()
        audio_buffer_filtered = io.BytesIO()  # Añadir buffer para señal filtrada

        # Guardar las señales
        sf.write(audio_buffer_message, self.audio, self.fs, format='WAV')
        sf.write(audio_buffer_modulated, self.modulated_signal, self.fs, format='WAV')
        sf.write(audio_buffer_demodulated, self.demodulated_signal, self.fs, format='WAV')
        sf.write(audio_buffer_filtered, self.demodulated_signal, self.fs, format='WAV')  # Señal filtrada

        # Devolver los buffers para reproducción en Streamlit
        audio_buffer_message.seek(0)
        audio_buffer_modulated.seek(0)
        audio_buffer_demodulated.seek(0)
        audio_buffer_filtered.seek(0)  # Retorna también la señal filtrada

        return audio_buffer_message, audio_buffer_modulated, audio_buffer_demodulated, audio_buffer_filtered

    def plot_lpf_response(self):
        """Muestra la respuesta en frecuencia del filtro Pasa Bajas"""
        w, h = freqz(self.b_digital, self.a_digital, fs=self.fs)
        plt.figure()
        plt.plot(w, 20 * np.log10(abs(h)))
        plt.title('Respuesta en Frecuencia del Filtro Pasa Bajas')
        plt.xlabel('Frecuencia [Hz]')
        plt.ylabel('Amplitud [dB]')
        plt.grid()
        st.pyplot(plt)

    def plot_poles_zeros(self):
        """Muestra el diagrama de polos y ceros del filtro Pasa Bajas"""
        z, p, k = signal.tf2zpk(self.b_digital, self.a_digital)
        plt.figure()
        plt.scatter(np.real(z), np.imag(z), marker='o', label='Ceros')
        plt.scatter(np.real(p), np.imag(p), marker='x', label='Polos')
        plt.title('Diagrama de Polos y Ceros')
        plt.xlabel('Parte Real')
        plt.ylabel('Parte Imaginaria')
        plt.grid()
        plt.legend()
        st.pyplot(plt)




# Streamlit App
def main():
    # Simulación de Sistema Masa-Resorte-Amortiguador
    st.title("Simulación de Sistema Masa-Resorte-Amortiguador y su equivalente RLC")

    st.write("""
    ### Sistema Masa-Resorte-Amortiguador:
    Esta aplicación interactiva permite simular un sistema masa-resorte-amortiguador y su equivalente en términos de un circuito **RLC**.
    Use las barras deslizantes para ajustar los valores de la masa, amortiguamiento y constante del resorte y observar las respuestas del sistema.
    """)

    # Inputs de usuario con barras deslizantes
    m = st.slider("Masa (m) [kg]", min_value=0.1, max_value=10.0, value=1.0, step=0.1)
    c = st.slider("Amortiguamiento (c) [Ns/m]", min_value=0.0, max_value=10.0, value=0.5, step=0.1)
    k = st.slider("Constante del Resorte (k) [N/m]", min_value=0.1, max_value=20.0, value=2.0, step=0.1)

    # Calcular los valores equivalentes del circuito RLC
    L, R, C = calcular_equivalente_RLC(m, c, k)

    # Simulación cuando el usuario presiona el botón
    if st.button("Simular Sistema"):
        # Obtener las respuestas del sistema
        w, mag, phase, t_step, step_response, t_impulse, impulse_response = simular_sistema_masa_resorte(m, c, k)

        # Mostrar los resultados
        st.write(f"**Parámetros Ingresados:**")
        st.write(f"- Masa (m): {m} kg")
        st.write(f"- Amortiguamiento (c): {c} Ns/m")
        st.write(f"- Constante del Resorte (k): {k} N/m")

        # Mostrar los valores equivalentes del circuito RLC
        st.write("**Equivalente RLC:**")
        st.write(f"- Inductancia (L): {L:.2f} H")
        st.write(f"- Resistencia (R): {R:.2f} Ω")
        st.write(f"- Capacitancia (C): {C:.4f} F")

        # Graficar resultados
        graficar_resultados(w, mag, phase, t_step, step_response, t_impulse, impulse_response)

    # Descarga y reconocimiento de género musical desde YouTube
    st.title("Descarga y reconocimiento de género musical desde YouTube")

    # Ingresar enlace de YouTube y tiempos de segmentación
    link_audio = st.text_input("Ingrese el enlace de YouTube:", "https://www.youtube.com/watch?v=51cZsDsVOqc")
    tiempo_inicio = st.slider("Tiempo de inicio del segmento (segundos):", min_value=0, max_value=60, value=30)
    tiempo_fin = st.slider("Tiempo de fin del segmento (segundos):", min_value=1, max_value=120, value=120)
    frecuencia_deseada = st.slider("Frecuencia de muestreo deseada (Hz):", min_value=8000, max_value=48000, value=48000)

    # Crear una instancia del descargador de audio
    downloader = AudioDownloader(link=link_audio)

    if st.button("Descargar y procesar audio"):
        try:
            # Paso 1: Descargar y convertir el audio desde YouTube
            downloader.download_audio()

            # Paso 2: Cargar el archivo de audio y remuestrearlo si es necesario
            audio, fs = downloader.load_audio(target_fs=frecuencia_deseada)

            # Paso 3: Segmentar el audio basado en la selección del usuario
            segmento, fs = downloader.segment_audio(start_time=tiempo_inicio, end_time=tiempo_fin)

            # Guardar el segmento en un buffer temporal
            audio_buffer = io.BytesIO()
            sf.write(audio_buffer, segmento, fs, format='WAV')
            audio_buffer.seek(0)

            # Reproducir el fragmento de audio seleccionado
            st.audio(audio_buffer, format="audio/wav")

            # Cargar el modelo de canciones (contiene espectros de metal y reguetón)
            model_path = "modelo/reggaeton_vs_metal.pkl"
            my_model_loaded = joblib.load(model_path)
            recognizer = AudioRecognizer(my_model_loaded)

            # Paso 4: Procesar el segmento de audio
            processed_segment = recognizer.process_segment(segmento, fs)

            # Paso 5: Reconocer el género de la canción y las probabilidades
            genre, probabilities = recognizer.recognize_genre(processed_segment)

            # Mostrar el resultado
            st.write(f"La canción es más probable que sea del género: {genre}")
            st.write(f"Probabilidad de Metal: {probabilities[0] * 100:.2f}%")
            st.write(f"Probabilidad de Reguetón: {probabilities[1] * 100:.2f}%")

        except Exception as e:
            st.error(f"Error al procesar el audio: {e}")



    # Simulación de Modulación AM y Demodulación
    st.title("Simulación de Modulación AM y Demodulación")


    # Usa el segmento de audio descargado y procesado
    if 'segmento' not in locals():
        st.error("Debe descargar y procesar el audio antes de proceder con la modulación AM.")
        return

    # Parámetros de modulación
    mod_index = st.slider("Índice de modulación", min_value=0.1, max_value=2.0, value=1.0, step=0.1)
    carrier_freq = st.slider("Frecuencia de la portadora (Hz)", min_value=1000, max_value=48000, value=48000, step=1000)

    # Crear instancia del modulador
    modulator = AMModulator(audio=segmento, fs=fs, carrier_freq=carrier_freq, mod_index=mod_index)

    # Paso 1: Realizar la modulación AM
    modulated_signal = modulator.modulate_am()

    # Paso 2: Realizar la demodulación AM
    demodulated_signal = modulator.demodulate_am()

    # Paso 3: Mostrar gráficos de las señales
    st.write("### Gráficos de las señales en el tiempo:")
    modulator.plot_signals()

    # Mostrar señales en el dominio de la frecuencia
    st.write("### Espectro de frecuencia:")
    modulator.plot_frequency_domain(segmento, title="Espectro del Mensaje (m(t))")
    modulator.plot_frequency_domain(modulated_signal, title="Espectro de la Señal Modulada AM")
    modulator.plot_frequency_domain(demodulated_signal, title="Espectro de la Señal Demodulada AM")

    # Mostrar respuesta del filtro pasa bajas y polos/ceros
    st.write("### Respuesta del Filtro Pasa Bajas:")
    modulator.plot_lpf_response()

    st.write("### Diagrama de Polos y Ceros del Filtro Pasa Bajas:")
    modulator.plot_poles_zeros()

    # Guardar y reproducir las señales
    audio_buffer_message, audio_buffer_modulated, audio_buffer_demodulated, audio_buffer_filtered = modulator.save_audio_signals()

    # Reproducir las señales
    st.write("### Reproducción de la señal de mensaje (m(t)):")
    st.audio(audio_buffer_message, format="audio/wav")

    st.write("### Reproducción de la señal modulada AM:")
    st.audio(audio_buffer_modulated, format="audio/wav")

    st.write("### Reproducción de la señal demodulada AM (filtrada):")
    st.audio(audio_buffer_filtered, format="audio/wav")





if __name__ == "__main__":
    main()


Writing parcial3_dashboard.py


Como al momento de correr el script con Streamlit se requiere del archivo .py donde se encuentre el código, utilizamos %%writefile para crearlo. Posteriormente hacemos toda la configuración de Ngrok con el token y demás datos de protocolo.

In [6]:
token = '2loY5BE5ZxQ3TstzfOMcpiGVm5q_4wkwQQXBRgu5GmePE82kL' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [7]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run parcial3_dashboard.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://3ee7-35-221-194-49.ngrok-free.app
